In [ ]:
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy as np 
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from collections import Counter
from scipy.sparse import csr_matrix
import os

corpus = [
    'This is Term Frequency',
    'This is Document Frequency',
    'This is Inverse Document Frequency',
    'This is Term Frequency - Inverse Document Frequency'            
]

#text preprocessing

stop = set(stopwords.words('english'))#set of stop words
sno = nltk.stem.SnowballStemmer('english')#initialising the Snowball Stemmer

def cleanhtml(sentence): #function to clean the word of html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    return cleantext

def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[ ? | ! | \' | " | # ] ' , r'' , sentence)
    cleaned = re.sub(r'[ . | , | ) | ( | \ | /]' , r'', cleaned)                     
    return cleaned

#code for implementing step-by-step the checks mentioned in pre-processing
i = 0
str1 = ' '
final_string =[]
s = ''
for sent in corpus:
    filtered_sentence =[]
    #print(sent);
    sent = cleanhtml(sent) #remove HTML tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words) > 2)):
                if(cleaned_words.lower() not in stop):
                    s = (sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                else:
                    continue
                    
            else:
                continue
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of clean words
    final_string.append(str1)
    i += 1

CleanedText = final_string

unique_word = {" ".join (corpus).split()}

#TF
def TF(corpus, unique_word):
    tf_dict = {}
    corpusCount = len(corpus)
    for word, count in corpus.items():
        tf_dict[word] = count/float(corpusCount)
    return(tf_dict)

#IDF
def IDF(corpus, unique_words):
    idf_dict={}
    corpusCount=len(corpus)
    for i in unique_words:
        count=0
        for sen in corpus:
            if i in sen.split():
                count=count+1
            idf_dict[i]=1+(math.log((1+corpusCount)/(count+1))) #1+log(N/n)
            #adding 1 to avoiding divid by zero errors #SCIKIT FORMULA
    return idf_dict 

#TFDIF

def TFIDF(tfs,idfs):
    tfidf_dict={}
    for word, val in tfs.items():
        tfidf[word] = val*idfs[word]
    return(tfidf_dict)
    # it accepts only list of sentances
    
    #we need prepocessing text here
    def fit(dataset):    
        unique_words = set() # at first we will initialize an empty set
        # check if its list type or not
        if isinstance(dataset, (list,)):
            for row in dataset: # for each review in the dataset
                for word in row.split(" "): # for each word in the review. #split method converts a string into list of words
                    if len(word) < 2:
                        continue
                    unique_words.add(word)
            unique_words = sorted(list(unique_words))
            vocab = {j:i for i,j in enumerate(unique_words)}
        
            return vocab
        else:
            print("you need to pass list of sentance")
     
    #we need prepocessing text here
    def transform(dataset,vocab):
        rows = []
        columns = []
        values = []
        if isinstance(dataset, (list,)):
            for idx, row in enumerate(tqdm(dataset)): # for each document in the dataset
                # it will return a dict type object where key is the word and values is its frequency, {word:frequency}
                word_freq = dict(Counter(row.split()))
                # for every unique word in the document
                for word, freq in word_freq.items():  # for each unique word in the review.                
                    if len(word) < 2:
                        continue
                    # we will check if its there in the vocabulary that we build in fit() function
                    # dict.get() function will return the values, if the key doesn't exits it will return -1
                    col_index = vocab.get(word, -1) # retreving the dimension number of a word
                    # if the word exists
                    if col_index !=-1:
                        # we are storing the index of the document
                        rows.append(idx)
                        # we are storing the dimensions of the word
                        columns.append(col_index)
                        # we are storing the frequency of the word
                        values.append(freq)
            return csr_matrix((values, (rows,columns)), shape=(len(dataset),len(vocab)))
        else:
            print("you need to pass list of strings")  
    
        
        
